In [11]:
import pandas as pd
import sqlite3 as sq

In [12]:
# Conectamos con la base de datos chinook.db

connection = sq.connect("./data_dos/chinook_dos.db")

# Obtenemos un cursor que utilizaremos para hacer las queries

cursor_clase = connection.cursor()

In [13]:
def sql_query(query):

    # Ejecuta la query
    cursor_clase.execute(query)
                                    

    # Almacena los datos de la query 
    ans = cursor_clase.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor_clase.description]

    return pd.DataFrame(ans,columns=names)

In [14]:
def get_column_names(table_name):
    # Conectar a la base de datos
    connection = sq.connect("./data_dos/chinook_dos.db")
    cursor = connection.cursor()

    # Ejecutar PRAGMA table_info para obtener la información de las columnas
    cursor.execute(f'PRAGMA table_info({table_name});')
    columns_info = cursor.fetchall()

    # Cerrar la conexión
    connection.close()

    # Extraer y devolver los nombres de las columnas
    column_names = [col[1] for col in columns_info]
    return column_names


### Ejercicio: Consultas con `JOINs` en la base de datos Chinook

1. **INNER JOIN**:
   - Selecciona los nombres de las pistas (`tracks.Name`) y los nombres de los álbumes (`albums.Title`).
   - Filtra para mostrar solo las pistas que pertenecen a álbumes cuyo nombre contiene la palabra "Rock".

2. **LEFT JOIN**:
   - Selecciona los nombres de los empleados (`employees.LastName`) y los nombres de los clientes (`customers.LastName`) que han sido atendidos por esos empleados.
   - Asegúrate de incluir todos los empleados, incluso aquellos que no han atendido a ningún cliente.

3. **RIGHT JOIN**:
   - Selecciona los nombres de los géneros (`genres.Name`) y los nombres de las pistas (`tracks.Name`).
   - Incluye todos los géneros, incluso aquellos que no tienen ninguna pista asociada.

4. **FULL JOIN**:
   - Selecciona los nombres de los artistas (`artists.Name`) y los nombres de los álbumes (`albums.Title`).
   - Incluye todos los artistas, incluso aquellos que no tienen álbumes, y todos los álbumes, incluso aquellos que no tienen un artista asociado.

5. **CROSS JOIN**:
   - Selecciona todas las combinaciones posibles de los nombres de las pistas (`tracks.Name`) y los nombres de los medios (`media_types.Name`).

6. **Múltiples JOINs**:
   - Selecciona los nombres de los clientes (`customers.LastName`), los nombres de los empleados (`employees.LastName`) que los han atendido, y los nombres de las ciudades (`customers.City`) donde viven los clientes.

7. **JOIN con agregación**:
   - Selecciona los nombres de los géneros (`genres.Name`) y la cantidad de pistas (`COUNT(tracks.TrackId)`) que pertenecen a cada género.
   - Asegúrate de agrupar por el nombre del género.


In [16]:
# Selecciona los nombres de las pistas (`tracks.Name`) y los nombres de los álbumes (`albums.Title`).

query = '''
SELECT tracks.Name, albums.Title
FROM tracks
JOIN albums
ON tracks.AlbumId=albums.AlbumId
'''
sql_query(query)

,Name,Title
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Put The Finger On You,For Those About To Rock We Salute You
2,Let's Get It Up,For Those About To Rock We Salute You
3,Inject The Venom,For Those About To Rock We Salute You
4,Snowballed,For Those About To Rock We Salute You
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Respighi:Pines of Rome
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Monteverdi: L'Orfeo
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Mozart: Chamber Music


In [17]:
# Filtra para mostrar solo las pistas que pertenecen a álbumes cuyo nombre contiene la palabra "Rock".

query = '''
SELECT tracks.Name, albums.Title
FROM tracks
JOIN albums
ON tracks.AlbumId=albums.AlbumId
JOIN genres 
ON tracks.GenreId = genres.GenreId
WHERE genres.Name = 'Rock';
'''
sql_query(query)

,Name,Title
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Balls to the Wall,Balls to the Wall
2,Fast As a Shark,Restless and Wild
3,Restless and Wild,Restless and Wild
4,Princess of the Dawn,Restless and Wild
...,...,...
1292,Tease Me Please Me,20th Century Masters - The Millennium Collecti...
1293,Wind of Change,20th Century Masters - The Millennium Collecti...
1294,Send Me an Angel,20th Century Masters - The Millennium Collecti...
1295,I Guess You're Right,Every Kind of Light


In [21]:
# Selecciona los nombres de los empleados (`employees.LastName`) y los nombres de los clientes (`customers.LastName`) que han sido atendidos por esos empleados.
# Asegúrate de incluir todos los empleados, incluso aquellos que no han atendido a ningún cliente.

query = '''
SELECT employees.LastName AS EmployeeLastName, customers.LastName AS CustomerLastName
FROM employees
LEFT JOIN customers
ON employees.EmployeeId = customers.SupportRepId;

'''
sql_query(query)

,EmployeeLastName,CustomerLastName
0,Adams,None
1,Edwards,None
2,Peacock,Gonçalves
3,Peacock,Tremblay
4,Peacock,Almeida
...,...,...
59,Johnson,Murray
60,Johnson,Rojas
61,Mitchell,None
62,King,None


In [22]:
# Selecciona los nombres de los géneros (`genres.Name`) y los nombres de las pistas (`tracks.Name`).
# Incluye todos los géneros, incluso aquellos que no tienen ninguna pista asociada.

query = '''
SELECT genres.Name AS Genres, tracks.Name AS Tracks
FROM genres
RIGHT JOIN tracks
ON genres.GenreId = tracks.GenreId;

'''
sql_query(query)

,Genres,Tracks
0,Rock,For Those About To Rock (We Salute You)
1,Rock,Balls to the Wall
2,Rock,Fast As a Shark
3,Rock,Restless and Wild
4,Rock,Princess of the Dawn
...,...,...
3498,Classical,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...
3499,Classical,"String Quartet No. 12 in C Minor, D. 703 ""Quar..."
3500,Classical,"L'orfeo, Act 3, Sinfonia (Orchestra)"
3501,Classical,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


In [23]:
query = '''
SELECT genres.Name AS GenreName, tracks.Name AS TrackName
FROM genres
LEFT JOIN tracks ON genres.GenreId = tracks.GenreId;
'''
sql_query(query)

,GenreName,TrackName
0,Rock,For Those About To Rock (We Salute You)
1,Rock,Balls to the Wall
2,Rock,Fast As a Shark
3,Rock,Restless and Wild
4,Rock,Princess of the Dawn
...,...,...
3498,Classical,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...
3499,Classical,"String Quartet No. 12 in C Minor, D. 703 ""Quar..."
3500,Classical,"L'orfeo, Act 3, Sinfonia (Orchestra)"
3501,Classical,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


In [25]:
#  Selecciona los nombres de los artistas (`artists.Name`) y los nombres de los álbumes (`albums.Title`).
# Incluye todos los artistas, incluso aquellos que no tienen álbumes, y todos los álbumes, incluso aquellos que no tienen un artista asociado.

query = '''
SELECT artists.Name AS Artist, albums.Title AS Album
FROM artists
LEFT JOIN albums ON artists.ArtistId = albums.ArtistId
UNION
SELECT artists.Name AS Artist, albums.Title AS Album
FROM albums
LEFT JOIN artists ON albums.ArtistId = artists.ArtistId;
'''
sql_query(query)


,Artist,Album
0,A Cor Do Som,None
1,AC/DC,For Those About To Rock We Salute You
2,AC/DC,Let There Be Rock
3,Aaron Copland & London Symphony Orchestra,"A Copland Celebration, Vol. I"
4,Aaron Goldberg,Worlds
...,...,...
413,Xis,None
414,Yehudi Menuhin,Bartok: Violin & Viola Concertos
415,Yo-Yo Ma,Bach: The Cello Suites
416,Youssou N'Dour,None
